### PREPARATION

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
!pip install autokeras
!pip install tensorflow_addons

In [ ]:
import autokeras as ak

DATA_PATH = "/data/cicdata.csv"
LABEL_PATH = "/data/ciclabel.csv"

early_warning = pd.read_csv(DATA_PATH)
rotulos = pd.read_csv(LABEL_PATH)

init = 99
train = 560
test = 1484
cols = ["5_kurt_total_ips_origem", 
        "5_skw_total_ips_destino", 
        "5_voefficient_variation_total_pacotes"]

In [ ]:
x_train = early_warning[cols][init:train]
y_train = rotulos['maior_2'][init:train]

x_test = early_warning[cols][train:test]
y_test = rotulos['maior_2'][train:test]

x_train = x_train.fillna(-1)
x_test = x_test.fillna(-1)

### CREATE MODEL 

In [ ]:
input_layer = ak.TimeseriesInput(lookback=1)
lstm_layer = ak.RNNBlock(layer_type="lstm", num_layers=5)(input_layer)
dense_layer = ak.DenseBlock()(lstm_layer)
output_layer = ak.RegressionHead()(dense_layer)

am = ak.AutoModel(inputs=input_layer,
                  outputs=output_layer,
                  max_trials=300,
                  overwrite=False)

In [ ]:
history = am.fit(x_train, x_train, validation_split=0.3)

In [ ]:
am_exp = am.export_model()

SAVE_PATH = ""
try:
    am_exp.save(f"{SAVE_PATH}", save_format="tf")
except Exception:
    am_exp.save(f"{SAVE_PATH}.h5")